In [96]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")
import pandas as pd
import numpy as np
import warnings
import mlflow
from modeling.config import TRACKING_URI, EXPERIMENT_NAME
from pandas_profiling import ProfileReport
import torch

In [97]:
alert_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/alerts.json")

In [98]:
notification_df=pd.read_json("/Users/fortes/neuefische/Capstone-WIMT/data/notifications.json")

In [100]:
notification_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value
0,20210529,1622251203899000,notification_receive,283a1,label,-Maph6wn8GA3SjPIc9HA
1,20210529,1622257988561000,notification_receive,283a1,label,-Maq63vqDUxQRawQ_3Au
2,20210529,1622258235184001,notification_receive,283a1,label,-Maq7-D7bhcEaFBU1LDW
3,20210529,1622260301025000,notification_receive,283a1,label,-MaqEsbKAqlxGG-RupjR
4,20210529,1622293962198000,notification_receive,283a1,label,-MasFHCt_lTuNt_x16dH
...,...,...,...,...,...,...
2278146,20210210,1612909178334000,notification_receive,fff18,label,0NbaiCRr6xGFd2035-Gdlw
2278147,20210210,1612969431222000,notification_receive,fff18,label,EMDjKLFr6xGRbrsa0gDjhg
2278148,20210210,1612978431552000,notification_receive,fff18,label,YE2VDMZr6xGRbrsa0gDjhg
2278149,20210210,1612985245425000,notification_receive,fff18,label,YBu-7NVr6xGRbrsa0gDjhg


In [109]:
#!pip3 install transformers --no-input

from transformers import pipeline

nlp_pos_syntax = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-pos-syntax",
    tokenizer="mrm8488/bert-spanish-cased-finetuned-pos-syntax"
)

text = 'Considera retrasos por servicios de emergencia atendiendo incendio en Mesones y Av. 20 de Noviembre.'

nlp_pos_syntax(text)[1:len(nlp_pos_syntax(text))-1]

In [125]:
cause_effect_table= pd.pivot_table(alert_df, index=['cause'], columns='effect',aggfunc='count',fill_value=0,values="is_global", margins=True, margins_name= 'Total')

In [126]:
cause_effect_table

effect,,CirculationRestored,CirculationShutdown,Delays,Evacuation,FullCapacity,HighWaitingTime,InterimService,LaneReduction,RouteDetour,SecuritySpeed,SuspensionOfService,TrafficImpact,Total
cause,,,,,,,,,,,,,,
BrokenTrafficLight,0,0,0,23,0,0,0,0,2,0,0,0,22,47
BrokenVehicle,0,0,4,390,0,0,0,0,39,0,0,12,55,500
COVID19,0,0,7,1,0,5,0,0,0,0,0,18,1,32
Counterflow,0,0,0,2,0,0,0,0,0,0,0,0,2,4
CycleRide,0,0,4,3,0,0,0,0,0,0,0,0,16,23
Demonstration,1,321,1896,166,0,0,0,65,54,5,6,251,226,2991
EmergencyServices,2,1,20,860,0,0,0,0,208,0,0,4,253,1348
Event,0,2,4,0,0,0,0,0,2,0,3,5,0,16
Explosion,0,0,0,0,0,0,0,0,0,0,0,0,1,1


Dropping the rows that contain source in join_field

In [142]:
source_purge= notification_df[(notification_df["join_field"]=="source")].index
notification_df.drop(source_purge, axis=0, inplace=True)

Fixing the event_name column:

In [145]:
notification_df["event_name"].replace("notification_receive","notification_received", inplace=True)

In [146]:
notification_df["event_name"].replace("notification_open","notification_opened", inplace=True)

In [165]:
label_merger= notification_df.query("join_field == 'label'")
alert_id_merger= notification_df.query("join_field == 'alertId'")

In [184]:
alert_df

,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,GtvOEQAFZ0GtU6u4AXwvPg,"[{'latitude': -99.13399344738725, 'longitude':..."
1,-MUH7_dRaJMkhnxY9bHs,0BBrcVJ26xGCgpvcY4zoVw,2021-02-24 03:43:22+00:00,2021-02-24 03:58:39+00:00,2021-02-24 04:21:00+00:00,Fire,Delays,🔥 Considera retrasos por servicios de emergenc...,false,JfA8Bw8Zp024Kqu4AXiSpQ,"[{'latitude': -99.13399344738725, 'longitude':..."
2,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55+00:00,2021-03-31 22:54:19+00:00,2021-04-01 00:01:01+00:00,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,GtvOEQAFZ0GtU6u4AXwvPg,"[{'latitude': -99.13838681335923, 'longitude':..."
3,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55+00:00,2021-03-31 22:54:19+00:00,2021-04-01 00:01:01+00:00,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,NuuRQ2I1Q0a50Kv-AVKlLA,"[{'latitude': -99.13838681335923, 'longitude':..."
4,-MX9U6P85DXAwGIaDsgi,kIdXsXOS6xGH7OsWJh6kOw,2021-03-31 22:51:55+00:00,2021-03-31 22:54:19+00:00,2021-04-01 00:01:01+00:00,Fire,TrafficImpact,🔥 ¡Anticípate! Afectación vial por servicios d...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.13838681335923, 'longitude':..."
...,...,...,...,...,...,...,...,...,...,...,...
13537,-MZZdjyfbnztCccgqoa1,sEfkzvmp6xG9Us96D1NGhQ,2021-04-30 21:19:54+00:00,2021-04-30 21:32:24+00:00,2021-04-30 22:14:00+00:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Oh que la! Afectación vial por semáforos de...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.15358161404815, 'longitude':..."
13538,-MZor-DYgDMER9B2NtGD,QHbP5XKs6xGbFYutnY_XdQ,2021-05-04 00:51:44+00:00,2021-05-04 00:52:21+00:00,2021-05-04 01:45:01+00:00,BrokenTrafficLight,TrafficImpact,🚦 ¡Nos lleva el chanfle! Afectación vial por s...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.11364770788302, 'longitude':..."
13539,-MWizRKZ_YIPcMWzaqMP,QNJvrEGO6xGgXg3PRUP4fg,2021-03-26 14:43:48+00:00,2021-03-26 14:44:15+00:00,2021-03-26 16:01:01+00:00,BrokenTrafficLight,TrafficImpact,⏳ ¡No te esponjes! Hay retrasos por semáforos ...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -98.96204243026706, 'longitude':..."
13540,-MS3yFJ5NoOacwMyGxSs,kO_VUshg6xGXlUHhke6ddw,2021-01-27 17:51:46+00:00,2021-01-27 17:51:51+00:00,2021-01-27 19:01:00+00:00,BrokenTrafficLight,Delays,⏳ ¡Chin! Hay retrasos por semáforos descompues...,false,zCy9zG00HEqGeKu4AWZYNQ,"[{'latitude': -99.17786568559703, 'longitude':..."


In [166]:
merged_df_1= pd.merge(label_merger, alert_df, left_on="join_field", right_on="surrogate_id", how="left")
merged_df_2= pd.merge(alert_id_merger, alert_df, left_on="join_field", right_on="document_id", how="left")

In [175]:
merged_df= pd.concat([merged_df_1, merged_df_2], axis=0)

In [182]:
merged_df_1

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203062,20210210,1612909178334000,notification_received,fff18,label,0NbaiCRr6xGFd2035-Gdlw,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203063,20210210,1612969431222000,notification_received,fff18,label,EMDjKLFr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203064,20210210,1612978431552000,notification_received,fff18,label,YE2VDMZr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203065,20210210,1612985245425000,notification_received,fff18,label,YBu-7NVr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
merged_df_2

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622254038809000,notification_received,0f737,alertId,-Maps-egC8fFLZM4HR5g,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622260302438000,notification_received,0f737,alertId,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622254036776000,notification_received,baafe,alertId,-Maps-egC8fFLZM4HR5g,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622258237428000,notification_received,e8300,alertId,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622258577858000,notification_opened,e8300,alertId,null,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36488,20210706,1625581262819000,notification_received,e0b4d,alertId,-MdwBMI8mNXadCw7GwTc,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36489,20210706,1625584900674000,notification_received,e0b4d,alertId,-MdwPCoa1U3j5wMZ8J99,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36490,20210706,1625587521801000,notification_received,e0b4d,alertId,-MdwZE8hMtqpcYoaNhzM,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36491,20210706,1625528529057001,notification_received,427c1,alertId,-Mdt2BT-2y05YXyJctt2,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
merged_df

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36488,20210706,1625581262819000,notification_received,e0b4d,alertId,-MdwBMI8mNXadCw7GwTc,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36489,20210706,1625584900674000,notification_received,e0b4d,alertId,-MdwPCoa1U3j5wMZ8J99,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36490,20210706,1625587521801000,notification_received,e0b4d,alertId,-MdwZE8hMtqpcYoaNhzM,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
36491,20210706,1625528529057001,notification_received,427c1,alertId,-Mdt2BT-2y05YXyJctt2,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
merged_df_1

,event_date,event_timestamp,event_name,user_id,join_field,join_key_value,document_id,surrogate_id,created_at,published_at,closed_at,cause,effect,description,is_global,agency,area_of_effect_coordinates
0,20210529,1622251203899000,notification_received,283a1,label,-Maph6wn8GA3SjPIc9HA,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,20210529,1622257988561000,notification_received,283a1,label,-Maq63vqDUxQRawQ_3Au,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,20210529,1622258235184001,notification_received,283a1,label,-Maq7-D7bhcEaFBU1LDW,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
3,20210529,1622260301025000,notification_received,283a1,label,-MaqEsbKAqlxGG-RupjR,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
4,20210529,1622293962198000,notification_received,283a1,label,-MasFHCt_lTuNt_x16dH,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203062,20210210,1612909178334000,notification_received,fff18,label,0NbaiCRr6xGFd2035-Gdlw,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203063,20210210,1612969431222000,notification_received,fff18,label,EMDjKLFr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203064,20210210,1612978431552000,notification_received,fff18,label,YE2VDMZr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2203065,20210210,1612985245425000,notification_received,fff18,label,YBu-7NVr6xGRbrsa0gDjhg,NaN,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
merged_df["document_id"].isnull().count()

2239560